In [1]:
import io, os, sys, types
from IPython import get_ipython
from nbformat import current
from IPython.core.interactiveshell import InteractiveShell


def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path


class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = current.read(f, 'json')


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
            for cell in nb.worksheets[0].cells:
                if cell.cell_type == 'code' and cell.language == 'python':
                    # transform the input to executable Python
                    code = self.shell.input_transformer_manager.transform_cell(cell.input)
                    # run the code in themodule
                    exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod


class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

sys.meta_path.append(NotebookFinder())


/Users/shahelkhan/anaconda2/lib/python2.7/site-packages/nbformat/current.py:19: UserWarning: nbformat.current is deprecated.

- use nbformat for read/write/validate public API
- use nbformat.vX directly to composing notebooks of a particular version

  """)


In [2]:
# -*- coding: utf-8 -*-
"""
Created on Tue Aug 22 14:14:47 2018

@author: shahelkhan

Bic.py

Purpose:
    - Calculate the most appropriate number of components for the GMM process.
    - Do this by testing different training datasets selecting
    - loop over these 50 times and calculate a mean and standard deviation for the 

"""
# Import the various modules necessary
import time
import numpy as np
from sklearn import preprocessing
from sklearn import mixture
from sklearn import decomposition
from math import log10, floor

import Load
import Print

start_time = time.clock()

importing Jupyter notebook from Load.ipynb
importing Jupyter notebook from Print.ipynb
('Printing runtime = ', 0.5786349999999998, ' s')
('Load runtime = ', 0.20259099999999997, ' s')


In [3]:
def main(address, filename_raw_data, subsample_bic, repeat_bic, max_groups, grid_bic,\
         conc_bic, size_bic, n_dimen, fraction_nan_samples, fraction_nan_depths):
    
    bic_many = np.ones((repeat_bic,max_groups-1)) # Need to use (max_groups-1) as the bic runs from 1 to max_groups 
    n_lowest_array = np.zeros(repeat_bic)
    n_comp_array = None
    for i in range(0,repeat_bic):
        print("Starting ", i)
        bic = bic_oneRun(address, filename_raw_data, subsample_bic, repeat_bic, max_groups, grid_bic,\
                   conc_bic, size_bic, n_dimen, fraction_nan_samples, fraction_nan_depths)
        bic_many[i,:] = bic[0]
        n_lowest_array[i] = bic[1]
        if i == 0 :
            n_comp_array = bic[2]
        del bic
        print("finished ", i)
        
    
    # bic_many shape (repeat, n_comp_array)

    bic_stand = preprocessing.StandardScaler()
    bic_stand.fit(bic_many)
    bic_mean = bic_stand.mean_
    bic_stdev = np.sqrt(bic_stand.var_)

In [7]:
# Calculate the most appropriate number of components from the bic_scores
def round_sig(x, x_2, sig=2):
    return round(x, sig-int(floor(log10(abs(x_2))))-1)

    n_stand = preprocessing.StandardScaler()
    n_stand.fit(n_lowest_array.reshape(-1,1))
    n_mean = n_stand.mean_[0]
    n_stdev = np.sqrt(n_stand.var_[0])

    n_mean = round_sig(n_mean, n_stdev)
    n_stdev = round_sig(n_stdev, n_stdev)
    
    # Alternative way of calculating the minimum number of components
    min_index = np.where(bic_mean==bic_mean.min())[0]
    n_min = (n_comp_array[min_index])[0]

    # Print to file
    Print.printBIC(address, repeat_bic, bic_many, bic_mean, bic_stdev, n_mean, n_stdev, n_min)

In [12]:
def bic_oneRun(address, filename_raw_data, subsample_bic, repeat_bic, max_groups, grid_bic,\
         conc_bic, size_bic, n_dimen, fraction_nan_samples, fraction_nan_depths):

    # Load the training data
    lon_train, lat_train, Tint_train, varTrain_centre, Sint_train, varTime_train \
            = None, None, None, None, None, None
    lon_train, lat_train, Tint_train, varTrain_centre, Sint_train, varTime_train \
        = Load.main(address, filename_raw_data, None, subsample_bic, False,\
         False, grid_bic, conc_bic, None, None, None,\
         fraction_nan_samples, fraction_nan_depths, run_Bic=True)
        

    # Calculate PCA
    pca, X_pca_train = None, None
    pca = decomposition.PCA(n_components = n_dimen)     # Initialise PCA object
    pca.fit(varTrain_centre)                         # Fit the PCA to the training data
    X_pca_train = pca.transform(varTrain_centre) 
    del pca
    
    # Run BIC for GMM with different number of components
    # bic_values contains the array of scores for the different n_comp
    # n_lowest is the lowest n for each repeat
    # n_comp_array is from 0 to max_groups in integers
    bic_values, n_lowest, n_comp_array = None, None, None
    bic_values, n_lowest, n_comp_array = bic_calculate(X_pca_train, max_groups)

    return bic_values, n_lowest, n_comp_array

In [13]:
def bic_calculate(X, max_groups):
#    print("BIC X shape = ",X.shape)
    X = X.reshape(-1,1)
#    print("BIC X.reshape shape = ", X.shape)
    lowest_bic, bic_score = np.infty, []
    n_components_range = np.arange(1, max_groups)
    for n_components in n_components_range:
#        print("BIC n_comp = ",n_components)
        gmm = mixture.GaussianMixture(n_components = n_components, covariance_type = 'diag')
        gmm.fit(X)
        bic_score.append(gmm.bic(X))
        if bic_score[-1] < lowest_bic:
            lowest_bic = bic_score[-1]
            lowest_n = n_components
    bic_score = np.asarray(bic_score).reshape(1,-1)
    return bic_score, lowest_n, n_components_range

print('Bic runtime = ', time.clock() - start_time,' s')

('Bic runtime = ', 0.4057599999999999, ' s')
